In [ ]:
import awkward as ak
import os
import keras
import keras.layers as layers
import numpy as np
from Sum import Sum
import uproot
import nbimporter
import import_ipynb
import matplotlib.pyplot as plt
from hffrag import fixedbinning
from hffrag import binneddensity
from numpy.lib.recfunctions import structured_to_unstructured
import tensorflow as tf
import keras_tuner as kt
import DeepSetNeuralNet

In [ ]:
%matplotlib inline 

In [ ]:
#The data is being stored in a tree datastructure. 
#We access the charm root using this command
tree = uproot.open("hffrag.root:CharmAnalysis")

In [ ]:
MAXEVENTS = 1e20
MAXTRACKS = 32
LR = 1e-2
MASKVAL = -999

In [ ]:
#Read in the data from the root file
track_features = ["AnalysisTracks_pt","AnalysisTracks_eta","AnalysisTracks_phi","AnalysisTracks_z0sinTheta","AnalysisTracks_d0sig","AnalysisTracks_d0","AnalysisTracks_d0sigPV","AnalysisTracks_d0PV"]
jet_features = ["AnalysisAntiKt4TruthJets_pt", "AnalysisAntiKt4TruthJets_eta", "AnalysisAntiKt4TruthJets_phi",
                "AnalysisAntiKt4TruthJets_ghostB_pt", "AnalysisAntiKt4TruthJets_ghostB_eta","AnalysisAntiKt4TruthJets_ghostB_phi"]
features = tree.arrays(jet_features+track_features,entry_stop = MAXEVENTS)

In [ ]:
#Select the events of interest
events = features[ak.sum(features["AnalysisAntiKt4TruthJets_pt"] > 25000, axis = 1) > 0]

In [ ]:
#Displays the number of jets being trained on
jets = events[jet_features][:,0]
print("The number of jets to train on is: ", len(jets))

#Select tracks from the events
tracks = events[track_features]

#Match the tracks to the jets
matchedtracks = tracks[DeepSetNeuralNet.Match_Tracks(jets,tracks)]

#Pad and Flatten the data
matchedtracks = DeepSetNeuralNet.flatten(matchedtracks, MAXTRACKS)

# Identify the the bottom jets and their associated tracks
bjets = ak.sum(jets["AnalysisAntiKt4TruthJets_ghostB_pt"] > 5000, axis=1) > 0
jets = jets[bjets]
bhads_pt = jets["AnalysisAntiKt4TruthJets_ghostB_pt"][:, 0].to_numpy()
bhads_eta = jets["AnalysisAntiKt4TruthJets_ghostB_eta"][:,0].to_numpy()
bhads_phi = jets["AnalysisAntiKt4TruthJets_ghostB_phi"][:,0].to_numpy()
bhads = np.stack([bhads_pt,bhads_eta,bhads_phi],axis = -1)

print("There are {} outputs".format(np.shape(bhads)[1]))
matchedtracks = matchedtracks[bjets]
print("There are {} inputs".format(np.shape(matchedtracks)[1]))

#Transform the jet and tracks to unstructed data.
jets = structured_to_unstructured(jets[jet_features[:-3]])
matchedtracks = structured_to_unstructured(matchedtracks)

#Fix the angles
jets = DeepSetNeuralNet.pt_eta_phi_2_px_py_pz_jets(jets).to_numpy()
tracks_p = DeepSetNeuralNet.pt_eta_phi_2_px_py_pz_tracks(matchedtracks.to_numpy())
bhads = DeepSetNeuralNet.pt_eta_phi_2_px_py_pz_jets(bhads)
print(np.shape(tracks_p))
print(np.shape(matchedtracks[:, :, 3:]))
tracks = np.concatenate([tracks_p,matchedtracks[:,:,3:].to_numpy()],axis = 2)

In [ ]:
#Load the training and validation data
X_train = np.load("/home/physics/phujdj/DeepLearningParticlePhysics/TrainingAndValidationData/X_train_data")
X_valid = np.load("/home/physics/phujdj/DeepLearningParticlePhysics/TrainingAndValidationData/X_valid_data")
y_train = np.load("/home/physics/phujdj/DeepLearningParticlePhysics/TrainingAndValidationData/y_train_data")
y_valid = np.load("/home/physics/phujdj/DeepLearningParticlePhysics/TrainingAndValidationData/y_valid_data")

In [ ]:
#Create a Keras model that is a mirror image of the DeepSetNeuralNetwork to tune the hyperparameters of.
def model_builder(hp):
    """
    This function lays out the Deep Set Neural Architecture
    - A neural network is applied first to the tracks to extract information from the tracks.
    - This information produces an ensemble space which, the outputs of which are then summed to produce
        the inputs for the next layer
    - A neural network is then applied to the jet data obtained from the tracks. 
        To perform current univariate regression.
    """
    # Create the ranges of hyperparameters to explore
    dropouts = hp.Choice('dropout', [0.001,0.05,0.20,0.40,0.50,0.60,0.70])
    track_layer = hp.Choice('track_layers',[32,64,128,256,512])
    jet_layer = hp.Choice('jet_layers',[32,64,128,256,512])
    activation_func = hp.Choice('act_func',["relu","elu","selu"])
    Learning_rate = hp.Choice('learning_rate',[1e-6,1e-5,1e-4,1e-3,1e-2])
    regularizers = hp.Choice("regularizer", [1e-6,1e-5,1e-4,1e-3,1e-2,1e-1])
    dropout_frequency = hp.Choice("DropoutFrequency",[1,2,3,4])

    #Create the track and jet layers
    track_layers = [len(track_features)]+[track_layer,track_layer,track_layer,track_layer,track_layer]
    jet_layers = [jet_layer,jet_layer,jet_layer,jet_layer,jet_layer,jet_layer]

    #Set the number of targets being explored
    n_targets = 3
    
    #Follows the DeepSetNeural Architecture
    inputs = layers.Input(shape=(None, track_layers[0])) # Creates a layer for each input
    outputs = inputs  # Creates another layer to pass the inputs onto the ouputs
    outputs = layers.Masking(mask_value=MASKVAL)(outputs) # Masks the MASKVAl values

    counter = 0
    for nodes in track_layers[:-1]:
        #The first neural network is a series of dense layers and is applied to each track using the time distributed layer
        outputs = layers.TimeDistributed( 
            layers.Dense(nodes, activation=activation_func, kernel_initializer= "he_normal",kernel_regularizer = keras.regularizers.l2(regularizers)))(outputs) # We use relu and the corresponding he_normal for the activation function and bias initializer
        if counter % dropout_frequency == 0: # Every two layers apply a dropout
            outputs = layers.Dropout(dropouts)(outputs)
        else:
            counter += 1
        outputs = layers.BatchNormalization()(outputs) #Apply a batch norm to improve performance by preventing feature bias and overfitting

    outputs = layers.TimeDistributed(layers.Dense( 
        track_layers[-1], activation='softmax'))(outputs) # Apply softmax to ouput the results of the track neural network as probabilities
    outputs = Sum()(outputs) # Sum the outputs to make use of permutation invariance

    counter = 0
    for nodes in jet_layers: #Repeat of the track neural network without the need for the timedistributed layers
        outputs = layers.Dense(nodes, activation=activation_func, kernel_initializer= "he_normal",kernel_regularizer = keras.regularizers.l2(regularizers))(outputs)
        if counter % dropout_frequency == 0:
            outputs = layers.Dropout(dropouts)(outputs)
        else:
            counter += 1
        outputs = layers.BatchNormalization()(outputs)

    outputs = layers.Dense(n_targets+n_targets*(n_targets+1)//2)(outputs) # The output will have a number of neurons needed to form the mean covariance function of the loss func

    Model = keras.Model(inputs=inputs, outputs=outputs) #Create a keras model

    # Specify the neural network's optimizer and loss function
    Model.compile(
    optimizer=keras.optimizers.Nadam(learning_rate=Learning_rate,clipnorm = 1.0), # Optimizer used to train model
    metrics = [DeepSetNeuralNet.Normal_Accuracy_Metric], # Metric used to assess true performance of model
    loss=DeepSetNeuralNet.LogNormal_Loss_Function, #Loss function
    )

    return Model

In [ ]:
#Set up the hyperparameter
SEED = tf.random.set_seed(42) # Generate a random seed
max_trials = 15 # Set the number of trials
tuner = kt.RandomSearch(model_builder,
                        objective='val_loss',
                        seed=SEED,
                        overwrite=True,
                        max_trials=max_trials,
                        directory='/home/physics/phujdj/DeepLearningParticlePhysics',
                        project_name="DeepSetHyperTraining",
                        )


In [ ]:
#Create an early stoping to properly survey the values
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# Search the parameter space to obtain the best hyperparameter values
tuner.search(X_train, y_train, validation_data=(
    X_valid, y_valid), epochs=20, callbacks=[stop_early])


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=10)[0]
print(f"""
The hyperparameter search is complete. The optimal number of track layers is {best_hps.get('track_layers')}, the optimal number of jet layers is {best_hps.get('jet_layers')}, the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}, the optimal dropout rate is {best_hps.get('dropout')} and finally the optimal activation function is {best_hps.get('act_func')}
""")